In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
import prettytable
import geopandas as gpd
import fiona

In [68]:
msaData = pd.read_excel('../data.xlsx', sheet_name="msaData")
msaData = msaData.set_index(['msa'])
msaData = msaData.loc[msaData.index.isin(['Albany-Lebanon, OR','Anniston-Oxford, AL','Austin-Round Rock-Georgetown, TX'])]
msaData.reset_index(inplace=True)

In [77]:
msaShape = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [70]:
# msaShape = msaShape[['cbsa','geoid','msa','STATE','December_2020_LaborForce','December_2020_URate',
#              'NUMBER_AIRPORTS','AIRPORT','CODE','NS_DOMESTIC','NS_INTERNATIONAL','NS_TOTAL','geometry']]

In [71]:
msaData = pd.merge(msaData, msaShape, how='left', on=['msa','STATE'])

In [73]:
msaData.shape

(3, 77)

In [78]:
msaShape.shape

(381, 77)

In [75]:
msaData = msaData[['cbsa', 'geoid', 'msa', 'STATE', 'December_2020_LaborForce',
       'December_2020_URate', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [76]:
msaData.head()

,cbsa,geoid,msa,STATE,December_2020_LaborForce,December_2020_URate,NUMBER_AIRPORTS,AIRPORT,CODE,NS_DOMESTIC,NS_INTERNATIONAL,NS_TOTAL,Avg. Local Sales Tax,State Sales Tax,Total Sales Tax,Property Tax per Capita,Property Tax Rank,Property Tax Throw Back,Taxable Wages Rate,New Employers Rate,Taxable Wage Base,Payroll Tax Notes,Individual Tax Rate,Individual Tax Brackets,Graduated Individual Tax,Franchise Tax Rate,Corporate Tax Rate,Corporate Tax Brackets,Graduated Corporate Tax,STATE_CODB,Fortune 500,Fortune 1000,HOUSING_PERMITS1,HOUSING_PERMITS2,PERMIT_CHANGE,SCORE,TOTPOP_CY,TOTPOP_FY,POP_ANNUAL_GROWTH,MEDAGE_CY,MEDHHINC_CY,HOME_VALUE,COLLEGE_DEGREE,DIV_INDEX,URBAN_AREA,COMP_INDEX,GROCERY,HOUSING,UTILITIES,TRANSPORTATION,HEALTH_CARE,MISC,APT_RENT,PHYSICAL_SCIENCES_ENROLLMENT,BIO_LIFE_ENROLLMENT,BUSINESS_MGMT_ENROLLMENT,EDUCATION_ENROLLMENT,ENGINEERING_ENROLLMENT,LAW_ENROLLMENT,MEDICINE_ENROLLMENT,ENROLLMENT,COMPLETIONS,NUMBER_OF_COLLEGES,MSA_CODB,UNIT_LABOR_COST,ENERGY_COST,STATE_LOCAL_TAX,OFFICE_RENT,JAN_RAIN,JAN_SNOW,JAN_TEMP_MAX,JAN_TEMP_MIN,JULY_RAIN,JULY_SNOW,JULY_TEMP_MAX,JULY_TEMP_MIN,geometry
0,10540,4110540,"Albany-Lebanon, OR",OR,60096.0,6.0,0,None,None,0,0,0,0.00,0.00,0.00,1487.0,27.0,Yes,0.70 - 5.40%,2.1,42100,Includes special payroll tax offset: 0.09%,9.9,4,Yes,No entity-level tax measured by net worth/capi...,7.6,2,Yes,102,0,0,446.0,384.0,-0.139,NaN,131026.0,138123.0,1.06,41.0,55420.0,269787.0,0.209,35.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,103,96,92,58,9.7,11.2,42.0,30.0,0.9,0.0,81.0,49.0,POINT (-18.17322 8.41703)
1,11500,0111500,"Anniston-Oxford, AL",AL,44289.0,4.4,0,None,None,0,0,0,5.22,4.00,9.22,582.0,50.0,Yes,0.65 - 6.80%,2.7,8000,Includes 0.06% employment security assessment,5.0,3,Yes,"""Business privilege tax imposed at following g...",6.5,1,No,100,0,0,66.0,72.0,0.091,NaN,116592.0,115051.0,-0.27,40.0,48593.0,124618.0,0.187,46.5,Anniston-Calhoun County AL,83.4,87.4,59.4,126.6,87.5,81.5,88.5,669.0,70.0,446.0,991.0,1025.0,0.0,0.0,0.0,NaN,NaN,NaN,96,109,106,67,62,4.5,0.7,54.0,33.0,4.6,0.0,90.0,70.0,POINT (8.84361 -4.52126)
2,12420,4812420,"Austin-Round Rock-Georgetown, TX",TX,1267150.0,5.1,0,Austin,AUS,55,8,63,1.94,6.25,8.19,1872.0,13.0,NaN,0.31 - 6.31%,2.7,9000,NaN,0.0,1,No,"Revised franchise tax, otherwise known as marg...",0.0,1,No,92,1,2,17030.0,18145.0,0.065,100.0,2296794.0,2610839.0,2.60,34.0,78659.0,293890.0,0.455,73.6,Austin TX,101.8,92.2,109.8,95.0,89.6,108.6,103.2,1530.0,2553.0,7487.0,13958.0,3174.0,8485.0,1069.0,0.0,176444.0,30450.0,25.0,100,91,99,80,126,2.3,0.2,61.0,38.0,2.1,0.0,95.0,72.0,POINT (-0.96286 -8.45697)


In [91]:
data = pd.concat( [msaShape, msaData] ).reset_index( drop=True )

In [92]:
data.drop_duplicates(subset=['cbsa','geoid','msa','STATE'], keep='last', inplace=True)
data.sort_values( 'msa', inplace=True)

In [144]:
data = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [145]:
bls = pd.read_excel( '../msaBlsSum.xlsx', usecols="A,E,F", converters={'cbsa':str,'geoid':str})


In [146]:
bls.shape

(381, 3)

In [147]:
data.shape

(381, 77)

In [148]:
data = data[['cbsa', 'geoid', 'msa', 'STATE','NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [149]:
data.columns

Index(['cbsa', 'geoid', 'msa', 'STATE', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LI

In [150]:
data = pd.merge(data, bls, how='left', on='cbsa')

In [151]:
data = data[['cbsa', 'geoid', 'msa', 'STATE', 'January_2021_LaborForce',
       'January_2021_URate', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [152]:
crs = {'init': 'epsg:4326'}
data = gpd.GeoDataFrame(data, crs=crs, geometry=data['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [142]:
bls.head()

,cbsa,msa,STATE,January_2021_LaborForce,January_2021_URate
0,11500,"Anniston-Oxford, AL",AL,46374,4.5
1,12220,"Auburn-Opelika, AL",AL,76610,3.2
2,13820,"Birmingham-Hoover, AL",AL,552337,3.7
3,19300,"Daphne-Fairhope-Foley, AL",AL,95885,3.5
4,19460,"Decatur, AL",AL,74023,2.9


In [143]:
data.to_csv('deleteMe.csv', index=False)

In [157]:
data.isna().sum()

cbsa                              0
geoid                            17
msa                               0
STATE                             0
January_2021_LaborForce           0
January_2021_URate                0
NUMBER_AIRPORTS                   0
AIRPORT                           0
CODE                              0
NS_DOMESTIC                       0
NS_INTERNATIONAL                  0
NS_TOTAL                          0
Avg. Local Sales Tax              0
State Sales Tax                   0
Total Sales Tax                   0
Property Tax per Capita           1
Property Tax Rank                 1
Property Tax Throw Back          55
Taxable Wages Rate                0
New Employers Rate               74
Taxable Wage Base                 0
Payroll Tax Notes               152
Individual Tax Rate               0
Individual Tax Brackets           0
Graduated Individual Tax          0
Franchise Tax Rate                0
Corporate Tax Rate                0
Corporate Tax Brackets      

In [158]:
data.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")